# Setting up for Borůvka's algorithm

To implement Borůvka's algorithm, we'll need to count the components of a graph and label its vertices according to the componenet they belong. To count (and eventually label) the components of a graph we'll need to find which vertices in a graph are reachable from a given vertex.

To test our algorithms we'll use a graph with six vertices. The graph has two components. One component with vertex $\{2\}$ and one component with the remaining vertices $\{0,1,3,4,5\}$.

![](./SimpleGraph.png)

The adjacency matrix $G$ of the graph is shown below.


In [ ]:
_ = float('inf')

G = [
    [_, 1, _, 1, _, _],  # vertex 0's neighbors
    [1, _, _, 1, _, 1],  # vertex 1's neighbors
    [_, _, _, _, _, _],  # vertex 2's neighbors
    [1, 1, _, _, _, 1],  # vertex 3's neighbors
    [_, _, _, _, _, 1],  # vertex 4's neighbors
    [_, 1, _, 1, 1, _]  # vertex 5's neighbors
]

The **reachability** of a vertex $s$ is the set of all vertices that can be reached from $s$ following any available path. For example, vertex 5 in $G$ can be reached from vertex 1 either directly via the edge $(1,5)$ or via the path comprises the edges $(1,3)$ and $(3,5)$.

The main challenge, when traversing a graph is to be able to trace back out footsteps. That's because we often arrive a vertices with multiple edges and we can only follow one edge at a time. After we follow that edge to where ever it may lead, we need to come back to its starting vertex and explore the remaining edges.

Algorithmically, we need a data structure to keep track of what edges to pursue next. This can be a simple data structure and none is simpler than a humble array.


Consider for example the situation in below. We arrive at the green vertex and we have two choices to follow: either the edge to the blue vertex or the edge to the red one.

![](./rgb.png)

Let's say that we go to the red vertex. Once we find what's there, we need to return to our starting point and follow the edge to the blue vertex. How do we remember where to go next? We use a simple array as shown in the code below.


In [ ]:
def reachability_of(s: int, G: list[list[int]]) -> list[int]:
    """
    Compute the vertices reachable from a given starting vertex
    in a directed graph represented by an adjacency matrix.

    Parameters
    ----------
    s : int
        Starting vertex index.
    G : list[list[int]]
        Adjacency matrix representation of the graph, where
        G[i][j] is the edge weight (or a sentinel `no_edge` value).

    Returns
    -------
    list[int]
        Vertices reachable from `s`.
    """

    # Grab what the input graph uses to indicate absence of an edge.
    # We can always expect that for a non trivial graph, ie a graph
    # with one or more vertices, there will be at least one diagonal
    # elemennt (where the no-edge value is present), so we grab the
    # first element of the diagonal.
    no_edge: int = G[0][0]
    # Initialize a list to return
    reach: list[int] = []
    # List to remember which vertices to try next. Start from the
    # given vertex.
    visit_next: list[int] = [s]

    # Explore vertices that we need to visit next. The algorithm stops
    # when the list of vertices to visit next is empty.
    while visit_next:  # shorcut to while len(...) > 0
        # Grab some vertex that we wish to visit next
        v: int = visit_next.pop()
        # Check to see if the vertex we just grabed is already
        # in the list of vertices reachable from s.
        if v not in reach:
            # The vertex we just grabed is not in the list, of
            # vertices accessible from s so let's add it.
            reach.append(v)
            # Now, find all of the neighbors of the vertex we just
            # added to the reach list. We do this by considering
            # every vertex in the graph (loop with u) and asking
            # if there is an edge between u and v. If there is an
            # edge, we add u to the list of vertices to visit next.
            for u in range(len(G)):
                if G[v][u] != no_edge:
                    visit_next.append(u)
    # Done
    return reach

In [ ]:
def report_reachability(s: int, graph: list[list[int]]) -> None:
    """Helper method to produce nice reports from the reachability method."""
    print(
        f"\nThe set of vertices reachable from {s} is: {reachability_of(s, graph)}")


# Vertices to test. We expect the reachability of 2 to be the vertex itself and
# the reachability of 1 to be every vertex in the graph except for 2.
test_vertices: list[int] = [2, 1]

# Run the tests.
for test_vertex in test_vertices:
    report_reachability(test_vertex, G)

In the code above, we grab the next vertex to explore from array `visit_next` using Python's [`list.pop`](https://docs.python.org/3/tutorial/datastructures.html) method as follows:

```python
v = visit_next.pop()
```

By default, `pop()` obtains the last element of the invoking array. Together with how we add elements to the array, using `append`, the code above operates the array `visit_next` in LIFO mode, i.e., as a stack. For vertex `1`, we obtain the following set of reachable vertices:

```
[1, 5, 4, 3, 0]
```

We can change the behavior of `visit_next` by obtaining its first element every time with

```python
v = visit_next.pop(0)
```

This will operare the array in LIFO mode, i.e., a queue. Again, if we look for the set of vertices reachable from vertex `1` we will get:

```text
[1, 0, 3, 5, 4]
```

It's the same vertices, but in different order. (And since we use the term _set_, it's worth reminding ourselves that order in not important in sets). That's the main difference between a stack and a queue retrieval of vertices from `visit_next`. For all we care, we could be removing the middle element of the array with:

```python
v = visit_next.pop(len(visit_next)//2)
```

and the result would be the same set, just slightly different order:

```text
[1, 3, 0, 5, 4]
```

All things being equal, many programmers prefer the stack approach.


# How the stack stuck with us

Stacks are how recursion is implemented. And recursion is central to computer programming, tracing its origins to [Peano's axiomatic foundation of natural numbers](https://en.wikipedia.org/wiki/Peano_axioms) in the 19th century and later in the foundational work by Kurt Gödel, Alonzo Church, and Alan Turing in the 1930s. In programming, recursion is a way to repeat tasks -- List relied heavily on recursion because it lacks the looping capabilities we see in other languages.

Recursion is also an elegant way to describe some algorithsm. For example, the reachability of a vertex `s` is the union of reachabilities of all its neighbors. In the example graph, starting with vertex `1`, and using the notation $\mathcal{R}(u)$ to denote the set of vertices reachable from $u$, we can write:

\begin{align*}
\mathcal{R}(1) = & \mathcal{R}(0) \cup \mathcal{R}(3) \cup \mathcal{R}(5)
\end{align*}

Next we compute $\mathcal{R}(0)$, $\mathcal{R}(3)$, etc. There may be some circular references that could keep us going around for ever -- recursion needs a stoping mechanism, a _base case_. In this example, there are two base cases: when a vertex has been already visited and when it has no neighbors. Formally, we write:

\begin{equation*}
\mathcal{R}(u) = \begin{cases}
\varnothing & \text{if}\ s\ \text{already visited} \\
\{s\} & \text{if}\ s\ \text{has no neighbors} \\
\{s\}\cup \bigcup\_{v\in \text{neighbors of}\ s} & \text{otherwise}
\end{cases}
\end{equation*}

The code implementation is given below.


In [ ]:
def recursive_reachability_of(s: int, G: list[list[int]], reach: list[int]) -> list[int]:
    """Recursive implementation of the reachability function"""
    # Grab the no edge sentinel from the given graph
    no_edge = G[0][0]

    # If vertex s is not the set of reachable vertices, add it. Else
    # return array reach as you receive it.
    if s not in reach:
        reach.append(s)
        # Now find all its neighbors and computer their reachability sets
        for u in range(len(G)):
            if G[s][u] != no_edge:
                recursive_reachability_of(u, G, reach)
    return reach


# Simple example with initial call using an empty array reachabiity
print(recursive_reachability_of(1, G, []))